In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
import os
import cv2
import glob
import easyocr

# Crear el lector de EasyOCR para el idioma adecuado
reader = easyocr.Reader(['en'])  

root_dir = "./datasets/matriculas-europeas"
subdirs = ["train", "val", "test"]

# Crear directorios de etiquetas si no existen
for subdir in subdirs:
    label_dir = os.path.join(root_dir, subdir, "labels")
    os.makedirs(label_dir, exist_ok=True)

# Función para obtener el cuadro delimitador de la matrícula usando EasyOCR
def detectar_matricula_easyocr(imagen):
    # Preprocesamiento de la imagen
    imagen_gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen_contrast = cv2.convertScaleAbs(imagen_gray, alpha=1.3, beta=15)  # Ajuste de contraste y brillo
    resultados = reader.readtext(imagen_contrast, detail=1)

    # Filtrar resultados para encontrar el texto más probable que sea una matrícula
    for (bbox, texto, _) in resultados:
        texto_limpio = ''.join(filter(str.isalnum, texto))  # Quitar espacios y caracteres especiales
        if 5 <= len(texto_limpio) <= 10:  # Ajuste según longitud esperada de una matrícula
            x_min = min(bbox[0][0], bbox[1][0], bbox[2][0], bbox[3][0])
            y_min = min(bbox[0][1], bbox[1][1], bbox[2][1], bbox[3][1])
            x_max = max(bbox[0][0], bbox[1][0], bbox[2][0], bbox[3][0])
            y_max = max(bbox[0][1], bbox[1][1], bbox[2][1], bbox[3][1])
            width = x_max - x_min
            height = y_max - y_min
            return x_min, y_min, width, height
    return None  

# Procesar cada conjunto de datos
for subdir in subdirs:
    img_dir = os.path.join(root_dir, subdir, "images")
    label_dir = os.path.join(root_dir, subdir, "labels")

    for img_path in glob.glob(f"{img_dir}/*.[jp][pn]g"):
        img_name = os.path.basename(img_path).split('.')[0]
        
        # Leer la imagen
        img = cv2.imread(img_path)
        height, width, _ = img.shape

        # Intentar detectar matrícula usando EasyOCR
        ocr_box = detectar_matricula_easyocr(img)
        
        if ocr_box:
            # Obtener coordenadas y dimensiones normalizadas para formato YOLO
            x, y, w, h = ocr_box
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            width_norm = w / width
            height_norm = h / height
        else:
            # Si EasyOCR no detecta, intentar una zona común para la matrícula (parte baja de la imagen)
            print(f"No se detectó matrícula en {img_name}. Usando zona predeterminada.")
            x, y = int(width * 0.2), int(height * 0.75)  # Zona estimada para la matrícula
            w, h = int(width * 0.6), int(height * 0.2)
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            width_norm = w / width
            height_norm = h / height

        # Guardar el archivo de etiqueta en formato YOLO
        label_file = os.path.join(label_dir, f"{img_name}.txt")
        with open(label_file, 'w') as f:
            f.write(f"0 {x_center} {y_center} {width_norm} {height_norm}\n")

        print(f"Etiqueta generada para {img_name} en {label_file}")


In [ ]:
from ultralytics.models.yolo.detect import DetectionTrainer
import torch
from ultralytics import YOLO

# Verifica si se puede usar una GPU
print("GPU disponible: ", torch.cuda.is_available())

# Parámetros de entrenamiento para el modelo
args = dict(
    model="yolo11n.pt",  # Modelo YOLO preentrenado
    data="license_plates.yaml",  # Archivo de configuración de datos
    epochs=150,  # Aumentar el número de épocas
    batch=64,  # Tamaño del lote ajustado
    imgsz=640,  # Tamaño de imagen (ajustable según GPU)
    multi_scale=True,  # Habilita escalado múltiple para variar el tamaño de imagen
    workers=4,  # Número de trabajadores para el DataLoader
    device="cuda" if torch.cuda.is_available() else "cpu",  # Usar GPU si está disponible, de lo contrario CPU
    verbose=True,  # Mostrar progreso de entrenamiento
    lr0=0.001,  # Tasa de aprendizaje inicial ajustada
    weight_decay=0.001,  # Regularización L2
    patience=25,  # Early stopping después de 20épocas sin mejora
    augment=True,  # Habilitar aumento de datos
)

# Inicializa el entrenador y entrena el modelo
trainer = DetectionTrainer(overrides=args)  
trainer.train()

: 

In [6]:
model_plate = YOLO('./runs/detect/train15')

In [6]:
import os
import cv2
from PIL import Image
import easyocr
import matplotlib.pyplot as plt

# Función para mostrar la imagen
def mostrar_imagen(imagen):
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

# Inicializar el lector OCR de EasyOCR
lector = easyocr.Reader(["en"], gpu=True)

def postprocesar_imagen(imagen):
    if imagen is None or imagen.size == 0:
        return None
    if len(imagen.shape) < 2 or imagen.shape[0] == 0 or imagen.shape[1] == 0:
        print("Error: La imagen no tiene las dimensiones adecuadas.")
        return None
    imagen_blur = cv2.GaussianBlur(imagen.copy(), (1, 1), 0)
    imagen_contrast = cv2.convertScaleAbs(imagen_blur, alpha=1.1, beta=15)
    return imagen_contrast

def detectar_texto(imagen_procesada):
    if imagen_procesada is None or imagen_procesada.size == 0:
        return imagen_procesada, "", 0.01
    
    resultado = lector.readtext(imagen_procesada, detail=1)
    arriba_izq = (0, 0)
    abajo_der = (0, 0)
    count = 0
    texto, probabilidad = "", 0.01

    for bbox, texto, probabilidad in resultado:
        (arriba_izq_actual, _, abajo_der_actual, _) = bbox
        arriba_izq_actual = tuple([int(val) for val in arriba_izq_actual])
        abajo_der_actual = tuple([int(val) for val in abajo_der_actual])

        if count == 0:
            arriba_izq = arriba_izq_actual
            abajo_der = abajo_der_actual
            count += 1
        else:
            abajo_der = abajo_der_actual

    if not resultado:
        return imagen_procesada, "", 0.01

    # Recortar la región de interés con las validaciones para evitar errores
    height, width = imagen_procesada.shape[:2]
    arriba_izq = (max(0, arriba_izq[0]), max(0, arriba_izq[1]))
    abajo_der = (min(width, abajo_der[0]), min(height, abajo_der[1]))

    roi_plate = imagen_procesada[arriba_izq[1]:abajo_der[1], arriba_izq[0]:abajo_der[0]]
    
    return roi_plate, texto, probabilidad

def procesar_deteccion(imagen_procesada):
    if imagen_procesada is None or imagen_procesada.size == 0:
        print("Error: Imagen procesada es None o está vacía.")
        return None, "", 0.01

    # Inicializar variables con valores predeterminados
    imagen_anterior = imagen_procesada
    texto_anterior = ""
    probabilidad_anterior = 0.01

    resultado = detectar_texto(imagen_procesada)
    # Verificar si detectar_texto devolvió resultados válidos
    if resultado:
        imagen_actual, texto_actual, probabilidad_actual = resultado
    else:
        return imagen_anterior, texto_anterior, probabilidad_anterior  # Devolver los valores iniciales si no hay detección

    # Ciclo de detección y actualización de variables
    while probabilidad_actual > probabilidad_anterior:
        probabilidad_anterior = probabilidad_actual
        imagen_anterior = imagen_actual
        texto_anterior = texto_actual
        resultado = detectar_texto(imagen_anterior)
        if resultado:
            imagen_actual, texto_actual, probabilidad_actual = resultado
        else:
            break  # Salir del bucle si no se obtienen nuevos resultados válidos

    return imagen_anterior, texto_anterior, probabilidad_anterior


def preprocesar_imagen(imagen):
    if imagen is None or imagen.size == 0:
        return None
    if len(imagen.shape) < 2 or imagen.shape[0] == 0 or imagen.shape[1] == 0:
        print("Error: La imagen no tiene las dimensiones adecuadas.")
        return None
    imagen_blur = cv2.GaussianBlur(imagen, (5, 5), 0)
    imagen_gray = cv2.cvtColor(imagen_blur, cv2.COLOR_BGR2GRAY)
    imagen_contrast = cv2.convertScaleAbs(imagen_gray, alpha=1.1, beta=10)
    _, imagen_binaria = cv2.threshold(imagen_contrast, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return imagen_binaria

def procesar_imagen(imagen_procesada):
    if imagen_procesada is None or imagen_procesada.size == 0:
        print("Error: Imagen procesada es None o está vacía.")
        return None
    if len(imagen_procesada.shape) < 2 or imagen_procesada.shape[0] == 0 or imagen_procesada.shape[1] == 0:
        print("Error: La imagen no tiene las dimensiones adecuadas.")
        return None

    imagen_actual, texto_actual, probabilidad_actual = procesar_deteccion(imagen_procesada)
    probabilidad_anterior = 0
    imagen_anterior = ""
    texto_anterior = ""

    while probabilidad_actual > probabilidad_anterior:
        probabilidad_anterior = probabilidad_actual
        imagen_anterior = imagen_actual
        texto_anterior = texto_actual
        post_procesado = postprocesar_imagen(imagen_actual)
        
        # Agregar chequeo para evitar procesar una imagen nula
        if post_procesado is None:
            break
            
        imagen_actual, texto_actual, probabilidad_actual = procesar_deteccion(post_procesado)

    if probabilidad_anterior > 0.01 and 3 < len(texto_anterior) < 10:
        return imagen_anterior, texto_anterior, probabilidad_anterior
    else:
        return None

def OCR(imagen):
    if imagen is None or imagen.size == 0:
        print("Error: Imagen de entrada vacía.")
        return None
    if len(imagen.shape) < 2 or imagen.shape[0] == 0 or imagen.shape[1] == 0:
        print("Error: La imagen no tiene las dimensiones adecuadas.")
        return None
    if len(imagen.shape) < 3 or imagen.shape[2] != 3:
        print("Error: La imagen no tiene los canales de color necesarios.")
        return None
    imagen_preprocesada = preprocesar_imagen(imagen.copy())
    return procesar_imagen(imagen_preprocesada)

# Cargar la imagen y ejecutar OCR
image_path = "./5951FPF.png"
imagen = cv2.imread(image_path)

ocr_resultado = OCR(imagen)
if ocr_resultado:
    imagen_resultado, texto, probabilidad = ocr_resultado
    if not probabilidad:
        print("No se detectó ningún texto")
    else:
        print(f"Resultado: {texto}, Probabilidad: {probabilidad}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Resultado: 5951FPF, Probabilidad: 0.9998589687661023


In [8]:
import os
import torch
import cv2
import easyocr
import csv
import logging
from ultralytics import YOLO

# Configura el logger para suprimir mensajes
logging.getLogger('ultralytics').setLevel(logging.WARNING)

# Verifica si se puede usar una GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU disponible: ", torch.cuda.is_available())

# Configuración de OCR
reader = easyocr.Reader(['es'])

classNames = ["person", "bicycle", "car", "motorbike", "bus"]

datos = {name: {"from_front": [], "to_front": []} for name in classNames}

# Cargar los modelos YOLO
model_general = YOLO('yolo11n.pt')  # Modelo para detectar personas y vehículos
model_plate = YOLO('best.pt')  # Modelo para detectar matrículas

# Configuración del archivo de video y CSV
filename = "C0142.MP4"
output_video_filename = "resultado_deteccion.mp4"
csv_filename = "resultados.csv"

# Inicializar el video y el escritor
cap = cv2.VideoCapture(filename)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_video = cv2.VideoWriter(output_video_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

#detecta procedencia del objeto
def detectar_direccion(track_id, label_name, x, frame):
    if (
        track_id not in datos[label_name]["from_front"]
        and track_id not in datos[label_name]["to_front"]
    ):
        if label_name == "person" or label_name == "bicycle":
            if (x) < frame.shape[1] * 0.2 or (frame.shape[1]*0.7< (x) < frame.shape[1] * 0.95):
                datos[label_name]["to_front"].append(track_id)
            else:
                datos[label_name]["from_front"].append(track_id)
        else:
            if (x) < frame.shape[1] * 0.7:
                datos[label_name]["from_front"].append(track_id)
            elif (x) > frame.shape[1] * 0.9:
                datos[label_name]["to_front"].append(track_id)
        return ''
    elif track_id in datos[label_name]["to_front"]:
        return ''
    elif track_id in datos[label_name]["from_front"]:
        return ''
    

# Preparación del archivo CSV
with open(csv_filename, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['fotograma', 'tipo_objeto', 'confianza', 'identificador_tracking', 'x1', 'y1', 'x2', 'y2',
                        'matrícula_en_su_caso', 'texto_matricula'])

    # Variables de conteo
    conteo_clases = {name: 0 for name in classNames}

    # Procesar el video con detección general
    results = model_general.track(source=filename, show=True, stream=True, verbose=False)
    frame_count = 0

    # Almacenar matrículas y sus IDs para el seguimiento con su confianza
    matrículas_seguimiento = {}

    # Iterar sobre cada resultado por frame
    for frame_result in results:
        frame_count += 1

        frame = frame_result.orig_img

        # Imprimir fotograma actual y total de fotogramas
        print(f'Procesando fotograma {frame_count}')

        # Obtener detecciones y agregar track_id de cada objeto
        for box in frame_result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            score = box.conf[0].item()
            label = int(box.cls[0])
            track_id = int(box.id[0]) if box.id is not None else -1

            if score >= 0.4:  # Umbral de confianza
                # Verificar si el label es válido para YOLO
                if 0 <= label < len(classNames):
                    label_name = classNames[label]

                    # Aumentar el conteo para la clase correspondiente
                    if(detectar_direccion(track_id, label_name, x2, frame) == None): continue

                    # Dibujar el rectángulo y mostrar el ID de seguimiento en el objeto
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                    cv2.putText(frame, label_name, (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                    # Procesar solo si es un vehículo para buscar matrículas
                    if label_name in ["car", "bus", "motorbike"]: 
                        roi = frame[y1:y2, x1:x2]  # Región de interés (ROI) del vehículo

                        # Usar el modelo de matrículas para detectar matrículas en la ROI
                        plate_results = model_plate.predict(source=roi, show=False)  

                        # Procesar los resultados de detección de matrículas
                        for plate_result in plate_results:
                            for plate_box in plate_result.boxes:
                                plate_x1, plate_y1, plate_x2, plate_y2 = map(int, plate_box.xyxy[0])
                                plate_score = plate_box.conf[0].item()

                                if plate_score >= 0.3:  # Umbral de confianza para matrículas
                                    # Dibuja el rectángulo de la matrícula en el frame
                                    cv2.rectangle(frame, (plate_x1 + x1, plate_y1 + y1), (plate_x2 + x1, plate_y2 + y1), (0, 255, 0), 2)

                                    # Leer el texto de la matrícula con preprocesamiento
                                    roi_plate = roi[plate_y1:plate_y2, plate_x1:plate_x2]

                                    # Aplicar OCR a la ROI de la matrícula

                                    if roi_plate is None or roi_plate.size == 0 or len(imagen.shape) < 2 or imagen.shape[0] == 0 or imagen.shape[1] == 0:
                                        print("Error: La región de interés (ROI) de la matrícula está vacía.")
                                    else:
                                        ocr_results_plate = OCR(roi_plate)

                                    if ocr_results_plate:
                                        imagen_resultado, matricula_texto, ocr_confianza = ocr_results_plate

                                        # Almacenar solo la matrícula con mayor confianza
                                        if (track_id not in matrículas_seguimiento) or (ocr_confianza > matrículas_seguimiento[track_id][1]) or (abs(len(matrículas_seguimiento[track_id][0])-7) > abs(len(matricula_texto)-7)):
                                            matrículas_seguimiento[track_id] = (matricula_texto, ocr_confianza)
                                            print(f'Matrícula detectada en fotograma {frame_count}: {matricula_texto} con confianza {ocr_confianza}')

                                    # Obtener la matrícula de mayor confianza para el CSV
                                    texto_matricula = matrículas_seguimiento[track_id][0] if track_id in matrículas_seguimiento else "N/A"

                                    # Escribir en el archivo CSV
                                    csv_writer.writerow([frame_count, label_name, score, track_id, x1, y1, x2, y2,
                                                        "matrícula", texto_matricula])

                                    # Mostrar el texto de la matrícula en el video
                                    cv2.putText(frame, texto_matricula, (plate_x1 + x1, plate_y1 + y1 - 10), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                                    break  # Romper después de procesar la primera matrícula detectada
                        else:
                            # Si no se detectó ninguna matrícula
                            csv_writer.writerow([frame_count, label_name, score, track_id, x1, y1, x2, y2,
                                                "N/A", "N/A"])
                    else:
                        # Escribir en el archivo CSV sin matrícula para objetos que no son vehículos
                        csv_writer.writerow([frame_count, label_name, score, track_id, x1, y1, x2, y2,
                                            "N/A", "N/A"])

                    # Mostrar la matrícula correspondiente si ya fue detectada con el texto de mayor confianza
                    if track_id in matrículas_seguimiento:
                        cv2.putText(frame, matrículas_seguimiento[track_id][0], (x1, y1 - 40), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Guardar el frame procesado en el archivo de video
        out_video.write(frame)

# Liberar recursos
cap.release()
out_video.release()

# Mostrar conteo de clases

conteo_clases = 0
for object, data in datos.items():
    conteo_clases += len(data['from_front']) + len(data['to_front'])
    print(
        f"{object}: \n\t{len(data['from_front'])} vienen de frente, \n\t{len(data['to_front'])} vienen de atras, \n\ttotal: {len(data['from_front']) + len(data['to_front'])}"
    )
print("Conteo total de objetos: ", conteo_clases)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


GPU disponible:  False
WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'

Procesando fotograma 1
Procesando fotograma 2
Procesando fotograma 3
Procesando fotograma 4
Procesando fotograma 5
Procesando fotograma 6
Procesando fotograma 7
Procesando fotograma 8
Procesando fotograma 9
Procesando fotograma 10
Procesando fotograma 11
Procesando fotograma 12
Procesando fotograma 13
Procesando fotograma 14
Procesando fotograma 15
Procesando fotograma 16
Procesando fotograma 17
Procesando fotograma 18
Procesando fotograma 19
Procesando fotograma 20
Procesando fotograma 21
Procesando fotograma 22
Procesando fotograma 23
Procesando f

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

# Asegurarse de que las gráficas se muestren en el notebook
%matplotlib inline

# Leer el archivo CSV de resultados
results_csv_path = './runs/detect/train4/results.csv'
results_df = pd.read_csv(results_csv_path)

# Mostrar las primeras filas del DataFrame y las columnas disponibles
print(results_df.head())
print("Columnas disponibles:", results_df.columns)

# Verificar si las columnas necesarias existen
required_columns = ['epoch', 'val/box_loss', 'train/box_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
for col in required_columns:
    if col not in results_df.columns:
        print(f"Error: La columna {col} no existe en el DataFrame.")
        exit()

# Graficar las métricas de entrenamiento y validación
plt.figure(figsize=(10, 5))
plt.plot(results_df['epoch'], results_df['val/box_loss'], label='Validation Box Loss')
plt.plot(results_df['epoch'], results_df['train/box_loss'], label='Training Box Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Box Loss')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(results_df['epoch'], results_df['metrics/precision(B)'], label='Precision')
plt.plot(results_df['epoch'], results_df['metrics/recall(B)'], label='Recall')
plt.plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP50')
plt.plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP50-95')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.legend()
plt.title('Precision, Recall, and mAP Metrics')
plt.show()